# 9 Ensamblador

### 9.9 Objetivo

Ensamblar modelos YA construidos
Los modelos a ensamblar deben ser de la misma Modalidad
 * Este script está armado para Analista Sr
 * Este script corre unicamente en Google Cloud

## 9.9.1  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Aug 24 21:19:10 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656928,35.1,1439211,76.9,1431552,76.5
Vcells,1224999,9.4,8388608,64.0,1924961,14.7


In [3]:
require("data.table")

Loading required package: data.table



#### Parametros

Se debe poner en PARAM$modelos  la lista de modelos que se va a ensamblar, y deben ser por lo menos DOS

In [4]:
PARAM <- list()
PARAM$modelos <- c( "WF1236", "WF9856" )

PARAM$experimento <- 999001


#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("~/buckets/b1/exp")
experimento_folder <- paste0("EN", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("~/buckets/b1/exp/", experimento_folder ))

### 9.9.2   Ensamblado

In [6]:
# lectura del primer  predicion.txt
tb_prediccion  <-  fread( paste0( "~/buckets/b1/exp/", PARAM$modelos[1], "/prediccion.txt" ) )
tb_prediccion <- tb_prediccion[,c("numero_de_cliente", "prob"), with=FALSE]
setorder(tb_prediccion, numero_de_cliente)

In [7]:
buenos <- 1 
for( imodelo in seq(2, length(PARAM$modelos)) )
{
  tbl <- fread( paste0( "~/buckets/b1/exp/", PARAM$modelos[imodelo], "/prediccion.txt" ) )
  if( nrow(tbl) == nrow( tb_prediccion ) )
    {
      setorder(tbl, numero_de_cliente)
      tb_prediccion[, prob := prob + tbl$prob ]
      buenos <- buenos +1 
    }
}

# fui sumando, ahora divido por la cantidad para tener el promedio
tb_prediccion[, prob := prob / buenos ]

# grabo a disto, esto SI PUEDE ser reutilizado
fwrite( tb_prediccion,
  file = "prediccion.txt",
  sep= "\t"
)

### 9.9.3 Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggles

In [8]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "data-mining-analista-jr-2025-a"
PARAM$kaggle$cortes <- seq(1800, 2400, by = 100)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0(
    "./kaggle/EN",
    PARAM$experimento, "_",
    envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios, "  Ensemble", "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")